In [1]:
import pandas as pd
from glob import glob
import numpy as np
from lr.text_processing.util import simple_pre_process_text_df
from lr.training.util import get_ternary_label, filter_df_by_label
from multiprocessing import Pool
from functools import reduce
from collections import Counter

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
syn_noum = pd.read_csv("data/snli/syn_noun.csv")
keys = syn_noum.key.to_list()
values = syn_noum.value.to_list()
syn_noum_dict = {k:v for k,v in zip(syn_noum.key, syn_noum.value)}
syn_noum_dict_inv = {v:k for k,v in zip(syn_noum.key, syn_noum.value)}

def check_key(s):
    return [w for w in s.split(" ") if w in keys] 

def check_key_df(df):    
    df.loc[:, "w_trans"] = df["text"].map(check_key)
    df.loc[:, "n_trans"] = df["w_trans"].map(lambda x : len(x))
    return df
    
def parallelize(df, func, n_cores):
    """
    general fucntion to parallelize a function applied to
    a df
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
df = pd.read_csv("data/snli/test.csv")
test = filter_df_by_label(df.dropna()).reset_index(drop=True)
df_t = pd.read_csv("data/snli/test_p_h_syn_noun.csv")
test_t = filter_df_by_label(df_t.dropna()).reset_index(drop=True)
test_t = test_t[["premise", "hypothesis"]].add_suffix("_t")

models = ["xgb", "albert_base", "bert_base", "roberta_base", "roberta_large", "xlnet_base"]

all_summaries ={}
all_res = {}

for model in models:

    all_combs = []
    paths = glob("raw_results/snli/{}/syn_p_h/batch1/*.csv".format(model))
    for path in paths:
        df1 = pd.read_csv(path).rename(columns={"label":"label_code"})
        assert test.shape[0]==test_t.shape[0]==df1.shape[0]
        comb = pd.concat([test,test_t, df1],1)
        comb_cut = comb.query("A==1 & B==0")
        all_combs.append(comb_cut)


    res = pd.concat(all_combs)
    res.index.name="id"
    res = res.reset_index()
    res = res.drop_duplicates(subset="id")
    summary = np.round((res["label"].value_counts()/res.shape[0])*100,1)
    all_res[model] = res
    all_summaries[model] = summary

In [4]:
for model in models:
    print(model)
    print(all_summaries[model])
    print()

xgb
entailment       34.9
neutral          34.2
contradiction    30.9
Name: label, dtype: float64

albert_base
contradiction    35.1
neutral          34.6
entailment       30.4
Name: label, dtype: float64

bert_base
neutral          41.8
entailment       32.9
contradiction    25.3
Name: label, dtype: float64

roberta_base
neutral          44.7
entailment       29.7
contradiction    25.6
Name: label, dtype: float64

roberta_large
neutral          44.2
entailment       32.2
contradiction    23.6
Name: label, dtype: float64

xlnet_base
neutral          40.0
entailment       31.3
contradiction    28.7
Name: label, dtype: float64



### Deep learning dificult

In [5]:
DL =  ["albert_base", "bert_base", "roberta_base", "roberta_large", "xlnet_base"]
count =[]
for model in DL:
    count += all_res[model].id.to_list()
    
count = Counter(count)


select = []
for k in count.keys():
    if count[k] == len(DL):
        select.append(k)

In [6]:
pct = len(select)/test.shape[0]
test.shape[0], len(select), "{:.1%}".format(pct)

(9824, 249, '2.5%')

In [9]:
pd.set_option("max_colwidth", None)
intesection_error = all_res["bert_base"].loc[all_res["bert_base"].isin({'id': select}).id]
intesection_error.head(20)

,id,premise,hypothesis,label,premise_t,hypothesis_t,label_code,A,B
1,31,3 young man in hoods standing in the middle of a quiet street facing the camera.,Three hood wearing people pose for a picture.,entailment,3 young adult male in hoods standing in the center of a quiet street facing the photographic camera,three punk wearing people pose for a image,1,1,0
2,32,3 young man in hoods standing in the middle of a quiet street facing the camera.,Three hood wearing people stand in a street.,entailment,3 young adult male in hoods standing in the center of a quiet street facing the photographic camera,three punk wearing people base in a street,1,1,0
3,44,Male in a blue jacket decides to lay in the grass.,The guy wearing a blue jacket is laying on the green grass taking a nap.,neutral,male in a blue jacket decides to ballad in the grass,the guy wearing a blue jacket is laying on the green grass taking a sleep,2,1,0
7,73,A man in a black shirt overlooking bike maintenance.,A man learns bike maintenance.,neutral,a adult male in a black shirt overlooking motorcycle care,a adult male acquire motorcycle care,2,1,0
8,78,A man looking over a bicycle's rear wheel in the maintenance garage with various tools visible in the background.,A man repairs bicycles.,entailment,a adult male looking over a bike rear wheel in the care garage with various tools visible in the background,a adult male repairs bike,1,1,0
13,118,Two teenage girls conversing next to lockers.,Girls talking next to the toilet.,contradiction,two teen girls conversing next to cabinet,girls talking next to the bathroom,0,1,0
37,406,A guy and a girl look down a mountain range.,The people look at the mountains.,entailment,a guy and a girl look down a mount scope,the people look at the mountains,1,1,0
41,499,a climber wearing a red headband is pulling himself up some gray rocks high above some green foliage.,A woman climber attempts to pull herself up,contradiction,a climber wearing a red headband is pulling himself up some grey rocks high above some green leaf,a adult female climber attempts to pull herself up,0,1,0
43,556,Boys with their backs against an incoming wave.,A group of people play in the ocean.,neutral,boys with their backs against an incoming moving ridge,a group of people play in the ocean,2,1,0
44,574,A hockey player in helmet.,The players are taking rest.,contradiction,a field hockey participant in helmet,the players are taking balance,0,1,0


In [8]:
inter_summary = np.round((intesection_error["label"].value_counts()/intesection_error.shape[0])*100,1)
inter_summary

entailment       45.4
neutral          32.5
contradiction    22.1
Name: label, dtype: float64